#### Dataset Creation

In [1]:
from utils import set_seed
from data_loading.models_dataset import ArchiMateModelDataset, EcoreModelDataset

set_seed(42)

config_params = dict(
    # reload=True,
    min_enr = -1,
    min_edges = 10,
    language = 'en',
)
dataset_name = 'eamodelset'

dataset = ArchiMateModelDataset(dataset_name, **config_params)
# dataset = EcoreModelDataset('modelset', **config_params)

Loading eamodelset from pickle
Loaded eamodelset with 558 graphs
Loaded eamodelset with 558 graphs
Graphs: 558


In [2]:
from data_loading.graph_dataset import GraphEdgeDataset
import utils

utils.set_seed(42)

graph_data_params = dict(
    reload=True,
    test_ratio=0.2,
    add_negative_train_samples=True,
    neg_sampling_ratio=1,
    distance=0,
    random_embed_dim=1,
    use_attributes=True,
    use_edge_label=True,
    use_edge_types=True,
    use_node_types=True,
    use_special_tokens=True,
    # use_embeddings=True,
    # embed_model_name='bert-base-cased',
    # ckpt='results/eamodelset/lp/10_att_0_nt_0/checkpoint-177600',
    limit = -1,
)

print("Loading graph dataset")
graph_edge_dataset = GraphEdgeDataset(dataset, **graph_data_params)
print("Loaded graph dataset")

texts = graph_edge_dataset.get_link_prediction_texts()

Loading graph dataset
Using tokenizer
Loading tokenizer for bert-base-uncased


Embeddings graphs:   0%|          | 0/558 [00:00<?, ?it/s]

Creating graphs:   0%|          | 0/558 [00:00<?, ?it/s]

['AndJunction' 'ApplicationCollaboration' 'ApplicationComponent'
 'ApplicationEvent' 'ApplicationFunction' 'ApplicationInteraction'
 'ApplicationInterface' 'ApplicationProcess' 'ApplicationService'
 'Artifact' 'Assessment' 'BusinessActor' 'BusinessCollaboration'
 'BusinessEvent' 'BusinessFunction' 'BusinessInteraction'
 'BusinessInterface' 'BusinessObject' 'BusinessProcess' 'BusinessRole'
 'BusinessService' 'Capability' 'CommunicationNetwork' 'Constraint'
 'Contract' 'CourseOfAction' 'DataObject' 'Deliverable' 'Device'
 'DistributionNetwork' 'Driver' 'Equipment' 'Facility' 'Gap' 'Goal'
 'Grouping' 'ImplementationEvent' 'Junction' 'Location' 'Material'
 'Meaning' 'Node' 'OrJunction' 'Outcome' 'Path' 'Plateau' 'Principle'
 'Product' 'Representation' 'Requirement' 'Resource' 'Stakeholder'
 'SystemSoftware' 'TechnologyCollaboration' 'TechnologyEvent'
 'TechnologyFunction' 'TechnologyInteraction' 'TechnologyInterface'
 'TechnologyProcess' 'TechnologyService' 'Value' 'ValueStream'
 'WorkPack

Validating edge classes:   0%|          | 0/558 [00:00<?, ?it/s]

Train classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Test classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Number of classes in training set: 11
Number of classes in test set: 11
Train edge classes: {4: 12829, 7: 7122, 8: 5405, 2: 5711, 3: 7522, 5: 4441, 1: 3517, 0: 3948, 10: 4429, 9: 1192, 6: 686}
Test edge classes: {7: 1751, 4: 3063, 3: 1827, 5: 1021, 1: 881, 0: 1019, 9: 314, 2: 1450, 10: 1068, 6: 177, 8: 1352}
Loaded graph dataset


Getting edge_cls data:   0%|          | 0/558 [00:00<?, ?it/s]

Train Texts:  ['<node_label>PASS<edge_begin><edge_end><node_label>European Front office', '<node_label>Merchant portal data exposure<edge_begin><edge_end><node_label>MP Reconciliation status preparation', '<node_label>Download Acquiring Transaction of a merchant user<edge_begin><edge_end><node_label>Send Acq data access request to data storage location', '<node_label>c IAM<edge_begin><edge_end><node_label>Broadcast Merchant data to platforms', '<node_label>Create contract hierarchies<edge_begin><edge_end><node_label>Create PASS merchant hierarchy', '<node_label>Activate existing merchant for reconciliation<edge_begin><edge_end><node_label>Update contract hierarchies for reconciliation', '<node_label>c IAM<edge_begin><edge_end><node_label>Enable Acceptance and Acquiring data access in Acceptance applications', '<node_label>Transaction report (Acc & Acquiring)<edge_begin><edge_end><node_label>Reconciliation engine', '<node_label>Merchant Portal<edge_begin><edge_end><node_label>e Com Acc 

In [ ]:
from data_loading.graph_dataset import GraphNodeDataset

graph_data_params = dict(
    reload=True,
    test_ratio=0.2,
    distance=1,
    random_embed_dim=1,
    use_attributes=True,
    # use_node_types=True,
    use_edge_label=True,
    use_edge_types=True,
    use_special_tokens=True,
    # use_embeddings=True,
    # embed_model_name='bert-base-cased',
    # ckpt='results/eamodelset/lp/10_att_0_nt_0/checkpoint-177600',
    node_cls_label='layer',
    limit = 20,
)

    # graph_data_params = dict(
    #     distance=args.distance,
    #     reload=args.reload,
    #     test_ratio=args.test_ratio,
    #     use_attributes=args.use_attributes,
    #     use_node_types=args.use_node_types,
    #     use_edge_types=args.use_edge_types,
    #     use_edge_label=args.use_edge_label,
    #     use_special_tokens=args.use_special_tokens,
    #     no_labels=args.no_labels,
    #     node_cls_label=args.cls_label,
    #     use_embeddings=args.use_embeddings,
    #     embed_model_name=args.embed_model_name,
    #     ckpt=args.ckpt,
    # )


print("Loading graph dataset")
graph_node_dataset = GraphNodeDataset(dataset, **graph_data_params)
print("Loaded graph dataset")
data = graph_node_dataset.get_node_classification_texts()

Loading graph dataset
Using tokenizer
Loading tokenizer for bert-base-uncased


Creating Embeddings:   0%|          | 0/20 [00:00<?, ?it/s]

Creating graphs:   0%|          | 0/20 [00:00<?, ?it/s]

['ApplicationCollaboration' 'ApplicationComponent' 'ApplicationEvent'
 'ApplicationFunction' 'ApplicationInteraction' 'ApplicationInterface'
 'ApplicationProcess' 'ApplicationService' 'Artifact' 'Assessment'
 'BusinessActor' 'BusinessCollaboration' 'BusinessEvent'
 'BusinessFunction' 'BusinessInteraction' 'BusinessInterface'
 'BusinessObject' 'BusinessProcess' 'BusinessRole' 'BusinessService'
 'Capability' 'CommunicationNetwork' 'Constraint' 'Contract'
 'CourseOfAction' 'DataObject' 'Deliverable' 'Device'
 'DistributionNetwork' 'Driver' 'Equipment' 'Facility' 'Gap' 'Goal'
 'Grouping' 'ImplementationEvent' 'Junction' 'Location' 'Material'
 'Meaning' 'Node' 'Outcome' 'Path' 'Plateau' 'Principle' 'Product'
 'Representation' 'Requirement' 'Resource' 'Stakeholder' 'SystemSoftware'
 'TechnologyCollaboration' 'TechnologyEvent' 'TechnologyFunction'
 'TechnologyInteraction' 'TechnologyInterface' 'TechnologyProcess'
 'TechnologyService' 'Value' 'ValueStream' 'WorkPackage']
Setting num_nodes_ typ

Validating node classes:   0%|          | 0/20 [00:00<?, ?it/s]

Train classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60}
Test classes: {0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 25, 26, 27, 29, 33, 34, 37, 40, 41, 44, 47, 48, 49, 50, 51, 56, 57}
Number of classes in training set: 60
Number of classes in test set: 37
['application' 'business' 'implementation_migration' 'motivation' 'other'
 'strategy' 'technology']
Setting num_nodes_ layer 7


Validating node classes:   0%|          | 0/20 [00:00<?, ?it/s]

Train classes: {0, 1, 2, 3, 4, 5, 6}
Test classes: {0, 1, 2, 3, 4, 5, 6}
Number of classes in training set: 7
Number of classes in test set: 7
Edge Classes:  ['Access' 'Aggregation' 'Assignment' 'Association' 'Composition' 'Flow'
 'Influence' 'Realization' 'Serving' 'Specialization' 'Triggering']
Node label: type
Train Node classes: {7: 53, 16: 43, 10: 32, 51: 8, 50: 34, 3: 84, 1: 96, 6: 74, 25: 58, 5: 29, 9: 23, 19: 15, 34: 19, 33: 35, 40: 22, 47: 15, 49: 15, 57: 20, 17: 67, 56: 1, 29: 14, 22: 9, 18: 27, 55: 3, 21: 2, 4: 2, 36: 7, 42: 6, 37: 2, 46: 5, 45: 5, 13: 52, 0: 2, 2: 9, 15: 5, 41: 14, 44: 10, 8: 2, 26: 1, 12: 9, 14: 14, 11: 3, 20: 2, 24: 2, 28: 1, 30: 1, 54: 1, 31: 1, 59: 1, 58: 1, 52: 1, 39: 1, 27: 2, 43: 1, 53: 1, 35: 1, 60: 1, 38: 1, 23: 1, 32: 1}
Test Node classes: {7: 7, 10: 12, 16: 12, 51: 1, 11: 3, 6: 21, 25: 12, 3: 27, 1: 25, 57: 3, 50: 14, 40: 8, 34: 3, 18: 11, 19: 6, 29: 4, 47: 4, 17: 13, 49: 4, 9: 7, 0: 2, 33: 5, 13: 15, 5: 6, 15: 2, 44: 2, 41: 3, 22: 2, 8: 2, 27: 1

Getting node classification data:   0%|          | 0/20 [00:00<?, ?it/s]

Tokenizing data
['<node_label>POE <edge_begin>type:Flow<edge_end> <node_label>TESP Power BI\n<node_label>POE <edge_begin>type:Flow<edge_end> <node_label>PET', '<node_label>Spare Parts Contract', '<node_label>WIN <edge_begin>type:Flow<edge_end> <node_label>PET', '<node_label>Non-disclosure Agreement (NDA)', '<node_label>Letter of Intent (LOI)', '<node_label>Out of Scope: Financial Planning Tool', '<node_label>Memorandum of Understanding (Mo U)', '<node_label>PET <edge_begin>type:Flow<edge_end> <node_label>Lab Ops PTT\n<node_label>PET <edge_begin>type:Flow<edge_end> <node_label>POE', '<node_label>Client <edge_begin>type:Flow<edge_end> <node_label>WIN', '<node_label>TESP Power BI <edge_begin><edge_end> <node_label>PET']
['<node_label>Lab Ops PTT <edge_begin><edge_end> <node_label>TESP Power BI\n<node_label>Lab Ops PTT <edge_begin>type:Flow<edge_end> <node_label>Out of Scope: Financial Planning Tool', '<node_label>TESP <edge_begin>type:Flow<edge_end> <node_label>PET', '<node_label>Employme

In [20]:
import networkx as nx

G = nx.Graph()
G.add_node(1, label='A')
G.add_node(2, label='B')
G.add_node(3, label='C')
G.add_node(4, label='D')
G.add_node(5, label='E')
G.add_node(6, label='F')

G.add_edge(1, 2, label='1')
G.add_edge(1, 3, label='2')
G.add_edge(1, 6, label='5')
G.add_edge(2, 3, label='6')
G.add_edge(2, 5, label='8')
G.add_edge(2, 6, label='9')
G.add_edge(3, 4, label='10')
G.add_edge(3, 5, label='11')
G.add_edge(4, 5, label='13')
G.add_edge(4, 6, label='14')
G.add_edge(5, 6, label='15')


In [21]:
from sklearn.model_selection import train_test_split


train_nodes, test_nodes = train_test_split(
	list(G.nodes), 
	test_size=0.2, 
	shuffle=True, 
	random_state=42
)

print(train_nodes, test_nodes)
nx.set_node_attributes(G, {node: 0 for node in train_nodes}, 'masked')
nx.set_node_attributes(G, {node: 1 for node in test_nodes}, 'masked')
G.nodes(data=True)

[6, 3, 5, 4] [1, 2]


NodeDataView({1: {'label': 'A', 'masked': 1}, 2: {'label': 'B', 'masked': 1}, 3: {'label': 'C', 'masked': 0}, 4: {'label': 'D', 'masked': 0}, 5: {'label': 'E', 'masked': 0}, 6: {'label': 'F', 'masked': 0}})

In [22]:
from torch_geometric.transforms import RandomLinkSplit
import torch
from data_loading.data import GraphData
import numpy as np

edge_index = np.array(G.edges()).T
transform = RandomLinkSplit(
	num_val=0, 
	num_test=0.2, 
	add_negative_train_samples=True,
	neg_sampling_ratio=1,
	split_labels=True
)

train_data, _, test_data = transform(GraphData(
	edge_index=torch.tensor(edge_index), 
	num_nodes=G.number_of_nodes()
))
nx.set_edge_attributes(G, {tuple(edge): False for edge in train_data.pos_edge_label_index.T.tolist()}, 'masked')
nx.set_edge_attributes(G, {tuple(edge): True for edge in test_data.pos_edge_label_index.T.tolist()}, 'masked')
G.edges(data=True)

EdgeDataView([(1, 2, {'label': '1', 'masked': False}), (1, 3, {'label': '2', 'masked': False}), (1, 6, {'label': '5', 'masked': False}), (2, 3, {'label': '6', 'masked': False}), (2, 5, {'label': '8', 'masked': False}), (2, 6, {'label': '9', 'masked': True}), (3, 4, {'label': '10', 'masked': False}), (3, 5, {'label': '11', 'masked': True}), (4, 5, {'label': '13', 'masked': False}), (4, 6, {'label': '14', 'masked': False}), (5, 6, {'label': '15', 'masked': False})])